#### Copyright (C) 2017 The University of Sydney, Australia

 This program is free software; you can redistribute it and/or modify it under
 the terms of the GNU General Public License, version 2, as published by
 the Free Software Foundation.
    
 This program is distributed in the hope that it will be useful, but WITHOUT
 ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
 FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
 for more details.
    
 You should have received a copy of the GNU General Public License along
 with this program; if not, write to Free Software Foundation, Inc.,
 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301, USA.
 
 Authors: John Cannon and Simon Williams

## Calculate subducting rate of paleo rasters at each age by sampling reconstructed rasters along subduction zones
The notebook also outputs total subduction zone length.
Note that this computation is not always accurate if the
plate topology model is not clean in the sense that there are
duplicated lines (instead of single lines, each shared by two adjoining plates) – this is the case
for the published AREPS plate motion model by Muller et al. (2016)
and these bugs will be fixed in the future.
Subduction zone lengths from models with such topological artefacts
need to be computed using the workflow described here (which attempts to detect and remove duplicated lines):
https://zenodo.org/record/154001#.WhooR7RdIkR

In [ ]:
from __future__ import print_function
import math
import os
import pygplates
import sys
# Add directory containing the 'ptt' module (Plate Tectonic Tools) to the Python path.
from ptt import subduction_convergence

# Input rotation and topology files.
rotation_filename = '../data/Global_EarthByte_230-0Ma_GK07_AREPS.rot'
rotation_model = pygplates.RotationModel(rotation_filename)
topology_filenames = [
        '../data/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',
        '../data/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']

# Output file containing results at each reconstruction time.
output_filename = '../data/subducting_raster.txt'

# Base filename and extension of raster to sample.
raster_filename_base = '../data/CO2'
raster_filename_ext = 'nc'

# Define the time range.
# The reconstruction time range (topologies resolved to these times).
# Also used to get paleo raster filenames based on 'raster_filename_base'.
min_time = 0
max_time = 1

# Tessellate the subduction zones to 0.5 degrees.
tessellation_threshold_radians = math.radians(0.5)

In [ ]:
#
# Sample grids
#
# The method used here for sampling of masked (NaN) rasters is faster than using the 'raster_query' module.

try:
    from netCDF4 import Dataset as netcdf
except ImportError:
    from scipy.io import netcdf_file as netcdf
    print('Warning: NetCDF4 grids not supported ("netCDF4" Python module not found). '
          'Falling back to NetCDF3, rasters may fail to load.', file=sys.stderr)

import scipy.interpolate as spi
import numpy as np

def sample_grid_using_scipy(x,y,grdfile):
    
    data=netcdf(grdfile,'r')
    try:
        lon = np.copy(data.variables['x'][:])
        lat = np.copy(data.variables['y'][:])
    except:
        lon = np.copy(data.variables['lon'][:])
        lat = np.copy(data.variables['lat'][:])
    
    Zg = data.variables['z'][:]
    
    test = fill_ndimage(Zg)
    
    lut=spi.RectBivariateSpline(lon,lat,test.T)
    result = []
    for xi,yi in zip(x,y):
        result.append(lut(xi, yi)[0][0])
            
    return result


from scipy import ndimage as nd

def fill_ndimage(data,invalid=None):
    """Replace the value of invalid 'data' cells (indicated by 'invalid')
    by the value of the nearest valid data cell
    Parameters
    ----------
    data: numpy array of any dimension
    invalid: a binary array of same shape as 'data'. True cells set where data
    value should be replaced.
    If None (default), use: invalid = np.isnan(data)
    Returns
    -------
    Return a filled array.
    Credits
    -------
    http://stackoverflow.com/a/9262129
    """
    if invalid is None: invalid = np.isnan(data)
    ind = nd.distance_transform_edt(invalid, return_distances=False, return_indices=True)
    return data[tuple(ind)]

In [ ]:
# Each element of this list will be a tuple of values for a specific reconstruction time.
output_data = []

# Reconstruction times.
reconstruction_times = range(min_time, max_time + 1)

# Iterate over time steps
for reconstruction_time in reconstruction_times:
    
    # Use existing subduction convergence script to generate sample points along subduction zones at 'time'.
    subduction_convergence_data = subduction_convergence.subduction_convergence(
                rotation_model,
                topology_filenames,
                tessellation_threshold_radians,
                reconstruction_time)
    
    # Determine paleo raster filename.
    raster_filename = '{0}_{1}.{2}'.format(raster_filename_base, reconstruction_time, raster_filename_ext)
    
    # Sample raster/grid at subduction points.
    subduction_lons = [data[0] for data in subduction_convergence_data]
    subduction_lats = [data[1] for data in subduction_convergence_data]
    raster_values = sample_grid_using_scipy(subduction_lons, subduction_lats, raster_filename)
    
    #
    # Do something with subducting raster values. Here we calculate average raster value.
    #
    total_subducting_length_metres = 0.0
    total_subducting_raster_per_year = 0.0
    total_raster_value_times_subduction_length = 0.0
    total_raster_value_squared_times_subduction_length = 0.0
    total_convergence_normal_velocity_times_subduction_length = 0.0
    total_convergence_normal_velocity_squared_times_subduction_length = 0.0
    total_raster_value_times_convergence_normal_velocity_times_subduction_length = 0.0
    total_raster_value_squared_times_convergence_normal_velocity_squared_times_subduction_length = 0.0
    for subduction_point_index, raster_value in enumerate(raster_values):

        # If couldn't find a non-NaN raster value within 20 degrees then skip to next subduction point.
        if math.isnan(raster_value):
            continue

        # We've got some other subduction data besides lon/lat locations.
        subduction_convergence_item = subduction_convergence_data[subduction_point_index]

        #lon = subduction_convergence_item[0]
        #lat = subduction_convergence_item[1]
        convergence_velocity_magnitude_cm_per_yr = subduction_convergence_item[2]
        convergence_obliquity_degrees = subduction_convergence_item[3]
        #absolute_velocity_magnitude = subduction_convergence_item[4]
        #absolute_obliquity_degrees = subduction_convergence_item[5]
        subducting_length_degrees = subduction_convergence_item[6]
        #subducting_arc_normal_azimuth = subduction_convergence_item[7]
        #subducting_plate_id = subduction_convergence_item[8]
        #subduction_zone_plate_id = subduction_convergence_item[9]

        subducting_length_metres = (
            math.radians(subducting_length_degrees) * 1e3 * pygplates.Earth.mean_radius_in_kms)
        
        convergence_normal_velocity_metres_per_year = (
            # 1e-2 converts cm/y to m/y...
            1e-2 * math.fabs(convergence_velocity_magnitude_cm_per_yr) *
            # Negative convergence handled by cos(obliquity_angle)...
            math.cos(math.radians(convergence_obliquity_degrees)))

        total_subducting_length_metres += subducting_length_metres
        
        #
        # If raster is a thickness in metres then 'total_subducting_raster_per_year' has units:
        #
        #   m^3/y = m * m * m/y
        #
        # ...which is a total volume rate.
        #
        # Otherwise if the raster is a quantity per raster pixel such as 'kg/m^2'
        # then 'total_subducting_raster_per_year' has units:
        #
        #   kg/y = kg/m^2 * m * m/y
        #
        # ...which is a total mass rate.
        #
        total_subducting_raster_per_year += (
                raster_value * subducting_length_metres * convergence_normal_velocity_metres_per_year)

        total_raster_value_times_subduction_length += raster_value * subducting_length_metres
        total_raster_value_squared_times_subduction_length += raster_value * raster_value * subducting_length_metres
        
        total_convergence_normal_velocity_times_subduction_length += (
                convergence_normal_velocity_metres_per_year * subducting_length_metres)
        total_convergence_normal_velocity_squared_times_subduction_length += (
                convergence_normal_velocity_metres_per_year * convergence_normal_velocity_metres_per_year *
                subducting_length_metres)
        
        total_raster_value_times_convergence_normal_velocity_times_subduction_length += (
                raster_value * convergence_normal_velocity_metres_per_year * subducting_length_metres)
        total_raster_value_squared_times_convergence_normal_velocity_squared_times_subduction_length += (
                raster_value * raster_value *
                convergence_normal_velocity_metres_per_year * convergence_normal_velocity_metres_per_year *
                subducting_length_metres)

    #
    # Statistics.
    #
    # mean = M = sum(Ci * Xi) / sum(Ci)
    # std_dev  = sqrt[sum(Ci * (Xi - M)^2) / sum(Ci)]
    #          = sqrt[(sum(Ci * Xi^2) - 2 * M * sum(Ci * Xi) + M^2 * sum(Ci)) / sum(Ci)]
    #          = sqrt[(sum(Ci * Xi^2) - 2 * M * M * sum(Ci) + M^2 * sum(Ci)) / sum(Ci)]
    #          = sqrt[(sum(Ci * Xi^2) - M^2 * sum(Ci)) / sum(Ci)]
    #          = sqrt[(sum(Ci * Xi^2) / sum(Ci) - M^2]
    #
    # ...where N is total number of sample points, Xi are sample and Ci are weights.
    #
    
    mean_subducting_raster_value = total_raster_value_times_subduction_length / total_subducting_length_metres
    variance_subducting_raster_value = (total_raster_value_squared_times_subduction_length / total_subducting_length_metres -
                                       mean_subducting_raster_value * mean_subducting_raster_value)
    std_dev_subducting_raster_value = (math.sqrt(variance_subducting_raster_value)
                                      if variance_subducting_raster_value > 0.0 else 0.0)
    
    mean_convergence_normal_velocity_metres_per_year = (
            total_convergence_normal_velocity_times_subduction_length / total_subducting_length_metres)
    variance_convergence_normal_velocity_metres_per_year = (
            total_convergence_normal_velocity_squared_times_subduction_length / total_subducting_length_metres -
            mean_convergence_normal_velocity_metres_per_year * mean_convergence_normal_velocity_metres_per_year)
    std_dev_convergence_normal_velocity_metres_per_year = (math.sqrt(variance_convergence_normal_velocity_metres_per_year)
                                      if variance_convergence_normal_velocity_metres_per_year > 0.0 else 0.0)
    
    mean_raster_value_times_convergence_normal_velocity = (
            total_raster_value_times_convergence_normal_velocity_times_subduction_length /
            total_subducting_length_metres)
    variance_raster_value_times_convergence_normal_velocity = (
            total_raster_value_squared_times_convergence_normal_velocity_squared_times_subduction_length /
                total_subducting_length_metres -
            mean_raster_value_times_convergence_normal_velocity * mean_raster_value_times_convergence_normal_velocity)
    std_dev_raster_value_times_convergence_normal_velocity = (
            math.sqrt(variance_raster_value_times_convergence_normal_velocity)
            if variance_raster_value_times_convergence_normal_velocity > 0.0 else 0.0)
    
    # Add a tuple of data values.
    # Can be any number of elements (they'll get written as columns to the output text file in the same order).
    output_data.append((
            reconstruction_time,
            mean_subducting_raster_value,
            std_dev_subducting_raster_value,
            mean_convergence_normal_velocity_metres_per_year,
            std_dev_convergence_normal_velocity_metres_per_year,
            mean_raster_value_times_convergence_normal_velocity,
            std_dev_raster_value_times_convergence_normal_velocity,
            total_subducting_length_metres,
            total_subducting_raster_per_year))

    print ('reconstruction_time: ', reconstruction_time)
    print ('mean_subducting_raster_value: ', mean_subducting_raster_value)
    print ('std_dev_subducting_raster_value: ', std_dev_subducting_raster_value)
    print ('mean_convergence_normal_velocity_metres_per_year: ', mean_convergence_normal_velocity_metres_per_year)
    print ('std_dev_convergence_normal_velocity_metres_per_year: ', std_dev_convergence_normal_velocity_metres_per_year)
    print ('mean_raster_value_times_convergence_normal_velocity: ', mean_raster_value_times_convergence_normal_velocity)
    print ('std_dev_raster_value_times_convergence_normal_velocity: ', std_dev_raster_value_times_convergence_normal_velocity)
    print ('total_subducting_length_metres: ', total_subducting_length_metres)
    print ('total_subducting_raster_per_year: ', total_subducting_raster_per_year)


In [ ]:
# Write the output data to a text file.
with open(output_filename, 'w') as output_file:
    for output_line in output_data:
        # Each 'output_line' is a tuple of values for a specific reconstruction time.
        # Write the elements of the tuple separated by spaces (and ending with a newline).
        output_file.write(' '.join(str(item) for item in output_line) + '\n')